In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
import csv
import datetime
from bs4 import BeautifulSoup
import time
import unicodedata

In [2]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(r"C:\Users\yamada\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe",options = options)
driver.get("https://kym22-web.ofc.kobe-u.ac.jp/campussy/")

In [3]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [4]:
year_attendance = 2025 #受講年度

exclude_keywords = ["海事科学部"] # 除外したいキーワードをリスト化
stop_keyword = "教養教育院"  # クリックを停止する基準

# 親要素 <class="sub"> を取得
parent_elements = soup.find_all(class_="sub")

# ローマ数字を正規化する関数
def normalize_roman_numerals(text):
    return unicodedata.normalize('NFKC', text)

# 学部リンクを収集
filtered_elements = []
for parent in parent_elements:
    if "学部" in parent.text:
        links = parent.find_all(
            "a",
            string=lambda x: x and not any(keyword in x for keyword in exclude_keywords),
        )
        for link in links:
            # 教養教育院までのリンクのみ収集
            link_text_normalized = normalize_roman_numerals(link.text.strip())
            if link_text_normalized == stop_keyword:
                filtered_elements.append(link)
                break
            filtered_elements.append(link)
            
csv_file_name = "class_" + str(year_attendance) + ".csv"

# CSVファイルを開く
f = open(csv_file_name, "w", encoding="UTF-8", errors="ignore", newline="")
writer = csv.writer(f, lineterminator="\n")

try:
    # 学部ごとにクリック操作を実行
    for element in filtered_elements:
        # ページをリロードしてリンクを再取得
        driver.get("https://kym22-web.ofc.kobe-u.ac.jp/campussy/")
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "sub")))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        parent_elements = soup.find_all(class_="sub")
    
        # 現在のリンクを再度取得（BeautifulSoup から再検索）
        selenium_element = None
        for parent in parent_elements:
            if "学部" in parent.text:
                links = parent.find_all(
                    "a",
                    string=lambda x: x and element.text.strip() == x.strip(),
                )
                if links:
                    selenium_element = driver.find_element("xpath", f"//*[text()='{element.text.strip()}']")
                    break
        if selenium_element:
            # Selenium で対象リンクをクリック
            driver.execute_script("arguments[0].click();", selenium_element)
            
             # セレクトボックスが表示されるまで待機
            dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "nendo"))
            )
    
            select = Select(dropdown)
            select.select_by_value(str(year_attendance))  
    
            dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "nendo"))
            )
            driver.execute_script("arguments[0].onchange();", dropdown)
            print(f"Clicked: {element.text.strip()}")
    
          
            weekdays = ["1", "2", "3", "4", "5", "6", "7"]  # 月～日を表す値、集中講義が欲しければ8まで指定
            for day in weekdays:
                print(f"Fetching data for weekday: {day}")
                
                # 対象の曜日リンクをクリック
                try:
                    day_link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//a[contains(@onclick, \"SelectYobiCall('{day}')\")]"))
                    )
                    driver.execute_script("arguments[0].click();", day_link)
                    time.sleep(2)  # ページがリロードされるまで待機
                except TimeoutException:
                    print(f"曜日リンクが見つかりませんでした: {day}")
                    # リンクが見つからなくても次の処理に進む
                    pass
                
                # 現在の曜日内のすべてのページを処理
                while True:
                    # データ行を取得
                    rows = driver.find_elements(By.XPATH, "//tr[@onmouseout='TRMouseOut(this)']")
                    for row in rows:
                        cells = row.find_elements(By.XPATH, ".//td")
                        csvlist = [cell.text for cell in cells[1:]]
                        writer.writerow(csvlist)
                
                    # 次のページへ進む
                    try:
                        next_page = driver.find_element(By.XPATH, "//a[contains(@onclick, \"nextpage\")]")
                        driver.execute_script("arguments[0].click();", next_page)
                        time.sleep(2)  
                    except Exception:
                        print(f"曜日 {day} のデータ収集が終了しました。")
                        break
                        
finally:
    print("すべてのデータが保存されました。")
    f.close()
    driver.quit()



Clicked: 文学部
Fetching data for weekday: 1
曜日リンクが見つかりませんでした: 1
曜日 1 のデータ収集が終了しました。
Fetching data for weekday: 2
曜日 2 のデータ収集が終了しました。
Fetching data for weekday: 3
曜日 3 のデータ収集が終了しました。
Fetching data for weekday: 4
曜日 4 のデータ収集が終了しました。
Fetching data for weekday: 5
曜日 5 のデータ収集が終了しました。
Fetching data for weekday: 6
曜日 6 のデータ収集が終了しました。
Fetching data for weekday: 7
曜日 7 のデータ収集が終了しました。
Clicked: 国際人間科学部
Fetching data for weekday: 1
曜日リンクが見つかりませんでした: 1
曜日 1 のデータ収集が終了しました。
Fetching data for weekday: 2
曜日 2 のデータ収集が終了しました。
Fetching data for weekday: 3
曜日 3 のデータ収集が終了しました。
Fetching data for weekday: 4
曜日 4 のデータ収集が終了しました。
Fetching data for weekday: 5
曜日 5 のデータ収集が終了しました。
Fetching data for weekday: 6
曜日 6 のデータ収集が終了しました。
Fetching data for weekday: 7
曜日 7 のデータ収集が終了しました。
Clicked: 国際文化学部
Fetching data for weekday: 1
曜日リンクが見つかりませんでした: 1
曜日 1 のデータ収集が終了しました。
Fetching data for weekday: 2
曜日 2 のデータ収集が終了しました。
Fetching data for weekday: 3
曜日 3 のデータ収集が終了しました。
Fetching data for weekday: 4
曜日 4 のデータ収集が終了しました。
Fetching dat